### Team 2 initial data wrangling.

In this notebook we will try to obtain the waymo data in a way that it is easily processed and with which we can start our data analysis.

Example code obtained from: 
- https://colab.research.google.com/github/waymo-research/waymo-open-dataset/blob/master/tutorial/tutorial.ipynb#scrollTo=JPc-xBE6VMHi

It was adapted in order to work with local windows machine

You also need to follow:

- https://github.com/waymo-research/waymo-open-dataset
- https://github.com/yyuuliang/waymo-dataset-viewer (if you want to avoid bazel installation)

in order to get the waymo open dataset package.

In [10]:
import os
import imp
import tensorflow as tf # > 1.14
import math
import numpy as np
import itertools
import matplotlib.pyplot as plt
import cv2
from os import listdir
from os.path import isfile, join

# TODO: Change this to your own setting
os.environ['PYTHONPATH']='/env/python:~/github/waymo-open-dataset'
m=imp.find_module('waymo_open_dataset', ['.'])
imp.load_module('waymo_open_dataset', m[0], m[1], m[2])

from waymo_open_dataset.utils import range_image_utils
from waymo_open_dataset.utils import transform_utils
from waymo_open_dataset import dataset_pb2 as open_dataset

In [13]:
n = '04'
tf.enable_eager_execution()
path = 'data/training_00'+n+'/' # Change path of data if needed here.
files = [f for f in listdir(path) if isfile(join(path,f))]

for i in range(len(files)):
    file = path+files[i]
    dataset = tf.data.TFRecordDataset(file, compression_type='')
    # Load frames from dataset
    frames = []
    for data in dataset:
        frame = open_dataset.Frame()
        frame.ParseFromString(bytearray(data.numpy()))
        frames.append(frame)

    print("filename:", file, "Num of frames:", len(frames))

    # ["FRONT", "FRONT_LEFT", "SIDE_LEFT", "FRONT_RIGHT", "SIDE_RIGHT"]
    POS = 0

    # Read img from waymo img class into numpy arrays
    imgs = []
    for frame in frames:
      l = list(enumerate(frame.images))
      _, im = l[POS]
      img = tf.image.decode_jpeg(im.image)
      height, width, _ = img.shape
      size = (width,height)
      imgs.append(img.numpy())
    
    output = 'videos/training_00'+n+'/'+ 'training_00'+n+'_' +files[i].split('.')[0] + ".avi" # Change path of video results if needed here.
    fps = 10

    # Write a sequence of imgs to a video  
    out = cv2.VideoWriter(output, cv2.VideoWriter_fourcc(*'XVID'), fps, size)
    for img in imgs:
        # Get it in GBR instead of RGB
        gbr = img[...,::-1].copy()
        out.write(gbr)
    out.release()

filename: data/training_0004/segment-10498013744573185290_1240_000_1260_000.tfrecord Num of frames: 199
filename: data/training_0004/segment-10517728057304349900_3360_000_3380_000.tfrecord Num of frames: 199
filename: data/training_0004/segment-1051897962568538022_238_170_258_170.tfrecord Num of frames: 198
filename: data/training_0004/segment-10584247114982259878_490_000_510_000.tfrecord Num of frames: 198
filename: data/training_0004/segment-10588771936253546636_2300_000_2320_000.tfrecord Num of frames: 199
filename: data/training_0004/segment-10596949720463106554_1933_530_1953_530.tfrecord Num of frames: 199
filename: data/training_0004/segment-10599748131695282446_1380_000_1400_000.tfrecord Num of frames: 198
filename: data/training_0004/segment-10625026498155904401_200_000_220_000.tfrecord Num of frames: 198
filename: data/training_0004/segment-10664823084372323928_4360_000_4380_000.tfrecord Num of frames: 198
filename: data/training_0004/segment-10676267326664322837_311_180_331_1